# Nucleotide Sequence Alignment Tool — Demo

This notebook demonstrates the **SeqAlign** tool, which performs pairwise nucleotide sequence alignment using the **Smith-Waterman algorithm** with **affine gap penalties**.

### How It Works

1. **Parse** two DNA sequences from FASTA files
2. **Align** them using Smith-Waterman (local alignment) with configurable scoring
3. **Display** the best alignment with statistics

In [1]:
import sys
sys.path.insert(0, '..')

from seqalign import parse_fasta, smith_waterman, format_alignment

---
### Example 1: Basic Alignment with Default Parameters

We align two closely related DNA sequences (Human vs Mouse BRCA1 exon fragments) with the default scoring:
- Match: **+5**
- Mismatch: **-4**
- Gap opening: **-12**
- Gap extension: **-2**

In [2]:
# Parse FASTA files
header1, seq1 = parse_fasta('seq1.fasta')
header2, seq2 = parse_fasta('seq2.fasta')

print(f'Sequence 1: {header1}')
print(f'  Length: {len(seq1)} bp')
print(f'  First 60 bp: {seq1[:60]}...')
print()
print(f'Sequence 2: {header2}')
print(f'  Length: {len(seq2)} bp')
print(f'  First 60 bp: {seq2[:60]}...')

Sequence 1: Seq1_Human_BRCA1_exon
  Length: 186 bp
  First 60 bp: ATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAA...

Sequence 2: Seq2_Mouse_Brca1_exon
  Length: 186 bp
  First 60 bp: ATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAACGCTATGCAGAAA...


In [3]:
# Perform alignment with default parameters
result = smith_waterman(
    seq1, seq2,
    match=5,
    mismatch=-4,
    gap_open=-12,
    gap_ext=-2
)

print(format_alignment(result))

  LOCAL ALIGNMENT (Smith-Waterman, Affine Gap Penalties)

  Score: 894.0
  Identity: 97.8% (182/186)
  Gaps: 0/186
  Mismatches: 4/186
  Alignment length: 186

  Seq1 region: 1 – 186
  Seq2 region: 1 – 186

----------------------------------------------------------------------

  Seq1     1  ATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAA  60
              |||||||||||||||||||||||||||||||||||||||||||||||.||||||||||||
  Seq2     1  ATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAACGCTATGCAGAAA  60

  Seq1    61  ATCTTAGAGTGTCCCATCTGTCTGGAGTTGATCAAGGAACCTGTCTCCACAAAGTGTGAC  120
              ||||||||||||||||||||.|||||||||||||||||||||||||||||||||||||||
  Seq2    61  ATCTTAGAGTGTCCCATCTGGCTGGAGTTGATCAAGGAACCTGTCTCCACAAAGTGTGAC  120

  Seq1   121  CACATATTTTGCAAATTTTGCATGCTGAAACTTTCGCCACCACTGGAGGTCTGCACCATA  180
              ||||||||||||||||||||||||||||||||||.|||||||||||||.|||||||||||
  Seq2   121  CACATATTTTGCAAATTTTGCATGCTGAAACTTTTGCCACCACTGGAGATCTGCACCATA  180

  Seq1   181  T

---
### Example 2: Aligning Sequences with Gaps

Let's test with sequences where we expect the algorithm to introduce gaps.

In [4]:
# Sequences with insertions/deletions
seq_a = 'ATCGATCGATCGATCGATCG'
seq_b = 'ATCGATGATCGATCG'  # 3 bases deleted in the middle

print(f'Seq A: {seq_a} ({len(seq_a)} bp)')
print(f'Seq B: {seq_b} ({len(seq_b)} bp)')
print()

result_gap = smith_waterman(
    seq_a, seq_b,
    match=5,
    mismatch=-4,
    gap_open=-12,
    gap_ext=-2
)

print(format_alignment(result_gap))

Seq A: ATCGATCGATCGATCGATCG (20 bp)
Seq B: ATCGATGATCGATCG (15 bp)

  LOCAL ALIGNMENT (Smith-Waterman, Affine Gap Penalties)

  Score: 61.0
  Identity: 93.8% (15/16)
  Gaps: 1/16
  Mismatches: 0/16
  Alignment length: 16

  Seq1 region: 1 – 16
  Seq2 region: 1 – 15

----------------------------------------------------------------------

  Seq1     1  ATCGATCGATCGATCG  16
              |||||| |||||||||
  Seq2     1  ATCGAT-GATCGATCG  15



---
### Example 3: Effect of Varying Gap Penalties

This example shows how different gap penalty settings affect the alignment. We use the same pair of sequences but vary the gap parameters.

In [5]:
seq_x = 'ACGTACGTAAAAACGTACGT'
seq_y = 'ACGTACGTACGTACGT'  # missing 'AAAA' in the middle

print('=== SCENARIO A: Harsh gap open penalty (gap_open=-16, gap_ext=-2) ===')
print('  (Discourages gaps → may prefer mismatches)\n')
result_a = smith_waterman(seq_x, seq_y, match=5, mismatch=-4, gap_open=-16, gap_ext=-2)
print(format_alignment(result_a))

print('\n=== SCENARIO B: Mild gap open penalty (gap_open=-4, gap_ext=-1) ===')
print('  (Gaps are cheap → algorithm freely introduces them)\n')
result_b = smith_waterman(seq_x, seq_y, match=5, mismatch=-4, gap_open=-4, gap_ext=-1)
print(format_alignment(result_b))

=== SCENARIO A: Harsh gap open penalty (gap_open=-16, gap_ext=-2) ===
  (Discourages gaps → may prefer mismatches)

  LOCAL ALIGNMENT (Smith-Waterman, Affine Gap Penalties)

  Score: 56.0
  Identity: 80.0% (16/20)
  Gaps: 4/20
  Mismatches: 0/20
  Alignment length: 20

  Seq1 region: 1 – 20
  Seq2 region: 1 – 16

----------------------------------------------------------------------

  Seq1     1  ACGTACGTAAAAACGTACGT  20
              ||||||||    ||||||||
  Seq2     1  ACGTACGT----ACGTACGT  16


=== SCENARIO B: Mild gap open penalty (gap_open=-4, gap_ext=-1) ===
  (Gaps are cheap → algorithm freely introduces them)

  LOCAL ALIGNMENT (Smith-Waterman, Affine Gap Penalties)

  Score: 72.0
  Identity: 80.0% (16/20)
  Gaps: 4/20
  Mismatches: 0/20
  Alignment length: 20

  Seq1 region: 1 – 20
  Seq2 region: 1 – 16

----------------------------------------------------------------------

  Seq1     1  ACGTACGTAAAAACGTACGT  20
              ||||||||    ||||||||
  Seq2     1  ACGTACGT----ACGT

---
### Example 4: Matching EBI LALIGN Parameters

To compare with [EBI LALIGN](https://www.ebi.ac.uk/jdispatcher/psa/lalign?stype=dna&gapext=0), use these parameters:
- Match: **+5**
- Mismatch: **-4**
- Gap opening: **-12**
- Gap extension: **0**

Submit the same sequences at the EBI LALIGN link and compare the results.

In [6]:
# Re-align with EBI LALIGN-like parameters (gap_ext = 0)
result_ebi = smith_waterman(
    seq1, seq2,
    match=5,
    mismatch=-4,
    gap_open=-12,
    gap_ext=0
)

print('Alignment with EBI LALIGN-matching parameters:')
print(f'  match=5, mismatch=-4, gap_open=-12, gap_ext=0\n')
print(format_alignment(result_ebi))

Alignment with EBI LALIGN-matching parameters:
  match=5, mismatch=-4, gap_open=-12, gap_ext=0

  LOCAL ALIGNMENT (Smith-Waterman, Affine Gap Penalties)

  Score: 894.0
  Identity: 97.8% (182/186)
  Gaps: 0/186
  Mismatches: 4/186
  Alignment length: 186

  Seq1 region: 1 – 186
  Seq2 region: 1 – 186

----------------------------------------------------------------------

  Seq1     1  ATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAA  60
              |||||||||||||||||||||||||||||||||||||||||||||||.||||||||||||
  Seq2     1  ATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAACGCTATGCAGAAA  60

  Seq1    61  ATCTTAGAGTGTCCCATCTGTCTGGAGTTGATCAAGGAACCTGTCTCCACAAAGTGTGAC  120
              ||||||||||||||||||||.|||||||||||||||||||||||||||||||||||||||
  Seq2    61  ATCTTAGAGTGTCCCATCTGGCTGGAGTTGATCAAGGAACCTGTCTCCACAAAGTGTGAC  120

  Seq1   121  CACATATTTTGCAAATTTTGCATGCTGAAACTTTCGCCACCACTGGAGGTCTGCACCATA  180
              ||||||||||||||||||||||||||||||||||.|||||||||||||.|||||||||||


---
### Example 5: Interactive Alignment (User Input)

Run the cell below — it will **prompt you** for two sequences and all four scoring parameters.
Simply type your values and press Enter. Press Enter without typing to use the default.

In [ ]:
# Interactive input for sequences
my_seq1 = input('Enter Sequence 1: ').strip().upper()
my_seq2 = input('Enter Sequence 2: ').strip().upper()

# Interactive input for scoring parameters
print('\n--- Scoring Parameters (press Enter for default) ---')
raw = input('  Match score [5]: ').strip()
my_match = float(raw) if raw else 5.0

raw = input('  Mismatch penalty [-4]: ').strip()
my_mismatch = float(raw) if raw else -4.0

raw = input('  Gap opening penalty [-12]: ').strip()
my_gap_open = float(raw) if raw else -12.0

raw = input('  Gap extension penalty [-2]: ').strip()
my_gap_ext = float(raw) if raw else -2.0

# Run alignment
print(f'\nAligning with: match={my_match}, mismatch={my_mismatch}, '
      f'gap_open={my_gap_open}, gap_ext={my_gap_ext}\n')

my_result = smith_waterman(
    my_seq1, my_seq2,
    match=my_match,
    mismatch=my_mismatch,
    gap_open=my_gap_open,
    gap_ext=my_gap_ext
)

print(format_alignment(my_result))


--- Scoring Parameters (press Enter for default) ---

Aligning with: match=2.0, mismatch=-1.0, gap_open=0.0, gap_ext=0.0

  LOCAL ALIGNMENT (Smith-Waterman, Affine Gap Penalties)

  Score: 16.0
  Identity: 80.0% (8/10)
  Gaps: 2/10
  Mismatches: 0/10
  Alignment length: 10

  Seq1 region: 1 – 9
  Seq2 region: 1 – 9

----------------------------------------------------------------------

  Seq1     1  ATC-GATCGA  9
              |||  |||||
  Seq2     1  ATCA-ATCGA  9



---
### Example 6: Interactive FASTA File Alignment (User Input)

Run the cell below — it will **prompt you** for paths to two FASTA files
and scoring parameters, then align and display the result.

In [ ]:
# Interactive input for FASTA file paths
file1 = input('Path to FASTA file 1: ').strip()
file2 = input('Path to FASTA file 2: ').strip()

h1, s1 = parse_fasta(file1)
h2, s2 = parse_fasta(file2)
print(f'  Seq1: {h1} ({len(s1)} bp)')
print(f'  Seq2: {h2} ({len(s2)} bp)')

# Interactive input for scoring parameters
print('\n--- Scoring Parameters (press Enter for default) ---')
raw = input('  Match score [5]: ').strip()
f_match = float(raw) if raw else 5.0

raw = input('  Mismatch penalty [-4]: ').strip()
f_mismatch = float(raw) if raw else -4.0

raw = input('  Gap opening penalty [-12]: ').strip()
f_gap_open = float(raw) if raw else -12.0

raw = input('  Gap extension penalty [-2]: ').strip()
f_gap_ext = float(raw) if raw else -2.0

# Run alignment
print(f'\nAligning with: match={f_match}, mismatch={f_mismatch}, '
      f'gap_open={f_gap_open}, gap_ext={f_gap_ext}\n')

result_file = smith_waterman(s1, s2, match=f_match, mismatch=f_mismatch,
                            gap_open=f_gap_open, gap_ext=f_gap_ext)
print(format_alignment(result_file))

  Seq1: Seq1_Human_BRCA1_exon (186 bp)
  Seq2: Seq2_Mouse_Brca1_exon (186 bp)

--- Scoring Parameters (press Enter for default) ---

Aligning with: match=2.0, mismatch=-1.0, gap_open=0.0, gap_ext=0.0

  LOCAL ALIGNMENT (Smith-Waterman, Affine Gap Penalties)

  Score: 364.0
  Identity: 95.8% (182/190)
  Gaps: 8/190
  Mismatches: 0/190
  Alignment length: 190

  Seq1 region: 1 – 186
  Seq2 region: 1 – 186

----------------------------------------------------------------------

  Seq1     1  ATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAA-TGCTATGCAGAA  59
              |||||||||||||||||||||||||||||||||||||||||||||||  |||||||||||
  Seq2     1  ATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTAAC-GCTATGCAGAA  59

  Seq1    60  AATCTTAGAGTGTCCCATCT-GTCTGGAGTTGATCAAGGAACCTGTCTCCACAAAGTGTG  118
              |||||||||||||||||||| | |||||||||||||||||||||||||||||||||||||
  Seq2    60  AATCTTAGAGTGTCCCATCTGG-CTGGAGTTGATCAAGGAACCTGTCTCCACAAAGTGTG  118

  Seq1   119  ACCACATATTTTGCAAATTTTGCATGCTGAAAC-TT